# Simple Neural Network Model with Homomorphic Encryption


# Build Neural Network (Without Encryption)

In [ ]:
import numpy as np

## Define the Perceptron Model
We start by defining the Perceptron class, which includes initializing weights, forward pass, and weight update methods.

In [ ]:

class Perceptron:
    def __init__(self, learning_rate=0.1, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = self._unit_step_func
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        y_ = np.array([1 if i > 0 else 0 for i in y])
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_func(linear_output)
                update = self.lr * (y_[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)
        return y_predicted

    def _unit_step_func(self, x):
        return np.where(x > 0, 1, 0)


## Create an Expanded Synthetic Dataset
Here, we'll create a larger dataset for our perceptron to classify.

In [23]:

from sklearn.model_selection import train_test_split

# Generate a synthetic dataset with more complexity
np.random.seed(0)
n_samples = 100
n_features = 10

X = np.random.randint(0, 2, size=(n_samples, n_features))
y = np.random.randint(0, 2, size=n_samples)  # Random binary labels

# Adjust the labels to depend on more complex logic (e.g., majority of the first three features)
y = (np.sum(X[:, :3], axis=1) > 1).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)
print(X_train[1])


[1 1 1 1 0 0 0 0 1 0]


## Train the Perceptron

In [24]:

p = Perceptron(learning_rate=0.1, n_iters=100)
p.fit(X_train, y_train)


## Evaluate the Model
We can now test the trained model with the test dataset.

In [25]:

predictions = p.predict(X_test)
print("Model predictions:", predictions)
print("Actual labels:", y_test)


Model predictions: [0 1 0 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0]
Actual labels: [0 1 0 1 0 1 1 1 1 0 1 1 1 0 0 0 0 1 1 0 1 0 0 1 0]


In [26]:
# Calculate accuracy
accuracy = np.mean(predictions == y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 84.00%


# Build Neural Network (With Encryption)

In [28]:
pip install tenseal

  Using cached tenseal-0.3.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.9 MB)


In [45]:
# Generate a context for the CKKS scheme
import tenseal as ts
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = 2**40
context.generate_galois_keys()

In [64]:
class EncryptedPerceptron:
    def __init__(self, context, learning_rate=0.1, n_iters=1000, n_features=None):
        self.context = context
        self.lr = learning_rate
        self.n_iters = n_iters
        self.n_features = n_features  # Number of features
        self.activation_func = self._unit_step_func
        self.weights = ts.ckks_vector(context, np.zeros(n_features))
        self.bias = ts.ckks_vector(context, [0])

    def fit(self, X, y):
        y_ = np.array([1 if i > 0 else 0 for i in y])

        for _ in range(self.n_iters):
            for idx, encrypted_x_i in enumerate(X):
                linear_output = encrypted_x_i.dot(self.weights) + self.bias
                y_predicted = self.activation_func(linear_output.decrypt())  # Decrypt on secure server

                update = self.lr * (y_[idx] - y_predicted)
                update_vector = update * np.array(encrypted_x_i.decrypt())  # Ensure the operation is on numpy array
                self.weights += ts.ckks_vector(self.context, update_vector)
                self.bias += ts.ckks_vector(self.context, [update])

    def predict(self, X):
        results = []
        for encrypted_x_i in X:
            linear_output = encrypted_x_i.dot(self.weights) + self.bias
            decrypted_output = linear_output.decrypt()  # Ensure decryption before the activation function
            y_predicted = self.activation_func(decrypted_output)
            results.append(y_predicted)
        return results

    def _unit_step_func(self, x):
        aggregate = sum(x)  # Sum up the decrypted list to get a single scalar value
        return 1 if aggregate > 0 else 0


In [70]:
from sklearn.model_selection import train_test_split

# Generate a synthetic dataset with more complexity
np.random.seed(0)
n_samples = 100
n_features = 10

X = np.random.randint(0, 2, size=(n_samples, n_features))
y = np.random.randint(0, 2, size=n_samples)  # Random binary labels

# Adjust the labels to depend on more complex logic (e.g., majority of the first three features)
y = (np.sum(X[:, :3], axis=1) > 1).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)
X_train_encrypted = [ts.ckks_vector(context, x) for x in X_train]
X_test_encrypted = [ts.ckks_vector(context, x) for x in X_test]

In [71]:
p = EncryptedPerceptron(context, learning_rate=0.1, n_iters=100, n_features=10)
p.fit(X_train_encrypted, y_train)
#Takes about 5 minutes to train

In [72]:
predictions = p.predict(X_test_encrypted)
print("Model predictions:", predictions)
print("Actual labels:", y_test)

Model predictions: [0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0]
Actual labels: [0 1 0 1 0 1 1 1 1 0 1 1 1 0 0 0 0 1 1 0 1 0 0 1 0]


In [73]:
# Calculate accuracy
accuracy = np.mean(predictions == y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 100.00%
